<a href="https://colab.research.google.com/github/diontimmer/so-vits-svc-merge/blob/main/so_vits_svc_merge_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# so-vits-svc-merge v1 by Dion Timmer
[(github)](https://github.com/diontimmer/)<br>
This simple function allows you to average the weights of two svc models creating a merged model and a (hopefully) awesome new voice.<br>
The ratio determines the amount of influence of model_b to model_a.

Model folders should be in the structure:<br>
<br>
**MODEL_FOLDER**<br>
└─── config.json<br>
└─── G_***.pth<br>


In [5]:
#@title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#@title Setup
import torch
import os

if not os.path.exists('merges'):
  os.makedirs('merges')

def ratio_merge(model_a, model_b, out_path = 'merges', speakername=None, alpha=0.5, device='cuda'):

    for file in os.listdir(model_a):
        if file.endswith('.pth') and file.startswith('G'):
            model_a_pth = os.path.join(model_a, file)
        elif file == 'config.json':
            model_a_config = os.path.join(model_a, file)

    for file in os.listdir(model_b):
        if file.endswith('.pth') and file.startswith('G'):
            model_b_pth = os.path.join(model_b, file)
        elif file == 'config.json':
            model_b_config = os.path.join(model_b, file)


    # Load Models
    model_0 = torch.load(model_a_pth, map_location=torch.device('cuda'))
    model_1 = torch.load(model_b_pth, map_location=torch.device('cuda'))
    theta_0 = model_0['model']
    theta_1 = model_1['model']

    # Merging Common Weights
    for key in theta_0.keys():
        if key in theta_1:
            theta_0[key] = (1 - alpha) * theta_0[key] + alpha * theta_1[key]

    # Merging Distinct Weights
    for key in theta_1.keys():
        if key not in theta_0:
            theta_0[key] = theta_1[key]

    if speakername is not None:
        out_path = os.path.join(out_path, speakername)
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        torch.save(model_0, f'{out_path}/G_merged.pth')
        # copy model_a_config json to out_path and change the "spk" dict to {SPEAKERNAME: 0}
        with open(model_a_config, 'r') as f:
            config = json.load(f)
            config['spk'] = {speakername: 0}
        with open(f'{out_path}/config.json', 'w') as f:
            json.dump(config, f, indent=4)

    return model_0

In [ ]:
#@title Parameters
model_a = "/content/drive/MyDrive/AI/Models/Drake" #@param {type:"string"}
model_b = "/content/drive/MyDrive/AI/Models/Juice" #@param {type:"string"}
out_path = 'merges' #@param {type:"string"}
speakername = 'DrakeJuice' #@param {type:"string"}
alpha = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
device = "cuda" #@param ["cuda", "cpu"]


In [ ]:
#@title Run
ratio_merge(model_a, model_b, out_path, speakername, alpha, device)
print('Process Finished!')